# 站間量分析

需要準備資料：
1. 票證資料：須包含所有佔位點的資料
2. 站序資料：需帶有'Direction'欄位
3. 班表資料：需帶有'Direction'、'IsWorkday'欄位
4. 營運月報 (optional)：做票證放大率佐證用 

## 基礎設定

包含環境設定，以及指定對應資料夾路徑（input、process、output）

In [1]:
import os
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display
from tickets_cleaning import tickets_cleaning, date_defined , getDaysCount, getMagnification

In [2]:
inputfolder_path = os.path.join(os.getcwd(),'..', 'input')
outputfolder_path = os.path.join(os.getcwd(),'..', 'output')
processfolder_path = os.path.join(os.getcwd(),'..', 'process')

# 確保資料夾存在
os.makedirs(inputfolder_path, exist_ok=True)
os.makedirs(outputfolder_path, exist_ok=True)
os.makedirs(processfolder_path, exist_ok=True)


In [3]:
date_turn_holiday=[20230929] # 補假、國定假日、颱風天
date_turn_workday=[20230923] # 補班
startdate = 20230701
enddate = 20230930
# 計算每月的假日與平日數
dayscount = getDaysCount(startdate, enddate, date_turn_holiday, date_turn_workday)


## 資料前處理

1. 票證清洗(去除不可用資料)
2. 票證定義日期欄位 (年月、平假日)
3. 處理票證放大率

In [4]:
'''進行基礎的票證清洗
1. 找到上車時間 < 下車時間
2. 上車站序 < 下車站序
3. 上下車站名不同'''

# 定義 tickets.csv 的相對路徑
tickets_path = os.path.join(inputfolder_path , 'tickets.csv')
tickets = pd.read_csv(tickets_path)

# 讀取資料並進行清理
tickets, errorstat, correctrate = tickets_cleaning(tickets, 
    getontime='GETON_DATE', 
    getofftime='GETOFF_DATE', 
    getonstop='GETON_STOP_NAME', 
    getoffstop='GETOFF_STOP_NAME', 
    getonseq='GETON_STOP_SEQ', 
    getoffseq='GETOFF_STOP_SEQ')

# 把清洗過的資料轉存至process
tickets.to_csv(os.path.join(processfolder_path , 'tickets_cleaned.csv'))
# tickets.to_csv(os.path.join(os.path.dirname(__file__), '..', 'process', 'tickets_cleaned.csv'))

# 輸出數據清洗統計
errorstat_path = os.path.join(outputfolder_path , 'ErrorDataStat.txt')
with open(errorstat_path , 'w', encoding='utf-8') as file:
    for key, value in errorstat.items():
        file.write(f"{key}: {value}\n")
# del errorstat
tickets = date_defined(tickets, getontime_columns='GETON_DATE', date_turn_holiday=date_turn_holiday,\
                       date_turn_workday=date_turn_workday)

# 針對這次的資料去做調整
tickets = tickets.rename(columns = {'ROUTE_NAME':'RouteName'})


C:\Users\kjchang\AppData\Local\Temp\ipykernel_28276\2479397280.py:8: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  tickets = pd.read_csv(tickets_path)


In [5]:
'''處理票證資料放大率'''

operation = pd.read_csv(os.path.join(inputfolder_path, 'operation.csv'))

# 計算 DataYearMonth 並格式化
operation['DataYearMonth'] = (
    pd.to_datetime((operation['YEAR'] + 1911) * 100 + operation['MONTH'], format='%Y%m')
    .dt.strftime('%Y%m')
)

tickets_magnification = getMagnification(
    tickets=tickets,
    tickets_routename_col='RouteName',
    tickets_yearmonth_col='DataYearMonth',  # 指定票證數據的年月欄位
    operation=operation,
    operation_routename_col='ROUTE_NAME',
    operation_yearmonth_col='DataYearMonth',  # 指定運營數據的年月欄位
    operation_passengers_col='PASSENGERS'
)

# 列出所有放大率會有異常的路線
ooc_route_list = list(set(tickets_magnification[tickets_magnification['Magnification'] >= 1.3]['RouteName'].unique()).union(
    set(tickets_magnification[tickets_magnification['Magnification'] <= 0.8]['RouteName'].unique())
))

### 基本判讀指標：是否繼續往下做

1. 列出本次資料正常資料的佔比
2. 列出本次放大率異常的路線 ( 可以進一步以plotly 圖表檢視長條圖)

In [9]:
print(f'資料可用比例 = {correctrate}%',end=' ')
if correctrate <= 95:
    print('本次取得的資料錯誤率太高，建議重新檢視')
else : 
    print('本次的資料可以使用')

try:
    if len(ooc_route_list) > 0:
        print(f'本次放大率異常路線共{len(ooc_route_list)}條')
        print('票證放大率異常的路線編號', end= ':')
        print(ooc_route_list)
except:
    pass

資料可用比例 = 99.5% 本次的資料可以使用
本次放大率異常路線共57條
票證放大率異常的路線編號:['L602A', 'L302B', 'L309B', 'L206A', 'L112', 'L309', 'L303A', 'L216', 'L721', 'L317A', 'L208', 'L725', 'L327', 'L115', 'L331A', 'L302', 'L116', 'L121', 'L319A', 'L306B', 'L218', 'L210', 'L728', 'L326', 'L723', 'L118', 'L722', 'L603B', 'L303', 'L317', 'L602B', 'L727', 'L716', 'L309A', 'L319', 'L517', 'L117', 'L113', 'L302A', 'L506', 'L209', 'L331D', 'L603A', 'L603', 'L306A', 'L717', 'L119', 'L211', 'L209A', 'L331C', 'L602', 'L215', 'L206', 'L718', 'L217', 'L207A', 'L120']


In [10]:
unique_year_months = tickets_magnification["DataYearMonth"].unique()
# 創建篩選器 (Dropdown)
dropdown = widgets.Dropdown(
    options=unique_year_months,
    value=unique_year_months[0],
    description="月份:"
)

# 定義繪圖函數
def plot_barchart(selected_month):
    # 篩選 DataFrame
    filtered_df = tickets_magnification[tickets_magnification["DataYearMonth"] == selected_month]
    
    if filtered_df.empty:
        print(f"No data available for {selected_month}")
        return
    
    # 創建條形圖
    fig = go.Figure()

    # 定義顯示在 hover 上的格式
    hover_text_tickets = [
    f"RouteName: {row['RouteName']}<br>Magnification: {row['Magnification'] * 100:.2f}%<br>Tickets: {row['Tickets']:,}"  # Magnification 顯示為百分比，Tickets 顯示為實際數字
    for _, row in filtered_df.iterrows()
    ]
    hover_text_passengers = [
    f"RouteName: {row['RouteName']}<br>Magnification: {row['Magnification'] * 100:.2f}%<br>Passengers: {row['Passengers']:,}"  # Magnification 顯示為百分比，Passengers 顯示為實際數字
    for _, row in filtered_df.iterrows()
    ]

    # 添加 Tickets 的長條圖
    fig.add_trace(go.Bar(
        x=filtered_df["RouteName"],
        y=filtered_df["Tickets"],
        name="Tickets",
        marker_color="#84C1FF",
        hovertext=hover_text_tickets,  # 顯示格式化過的 hovertext
        hoverinfo="text"  # 只顯示 hovertext 的內容
    ))

    # 添加 Passengers 的長條圖
    fig.add_trace(go.Bar(
        x=filtered_df["RouteName"],
        y=filtered_df["Passengers"],  # 更新欄位名稱為 Passengers
        name="Passengers",
        marker_color="#FF8000",
        hovertext=hover_text_passengers,  # 顯示格式化過的 hovertext
        hoverinfo="text"  # 只顯示 hovertext 的內容
    ))

    # 設定標題與軸標籤
    fig.update_layout(
        title=f"Tickets and Passengers for {selected_month}",
        xaxis_title="路線編號",
        yaxis_title="人次",
        barmode="group",  # 並列顯示長條圖
        xaxis_tickangle=-90,
        template="plotly_white"  # 使用白色背景的模板
    )

    # 顯示圖表
    fig.show()

# 綁定事件到篩選器
dropdown.observe(lambda change: plot_barchart(change.new), names="value")

# 初始顯示
display(dropdown)
plot_barchart(dropdown.value)


Dropdown(description='月份:', options=('202307', '202308', '202309'), value='202307')

In [11]:
# 確認班次資料皆有
shift = pd.read_excel(os.path.join(inputfolder_path, 'shift.xlsx'))
shift.columns = ['RouteName', 'Direction', 'Shift', 'IsWorkday']
shift['IsWorkday'] = shift['IsWorkday'].replace({'假日': '0', '平日': '1'})
shift = shift.sort_values(['RouteName', 'IsWorkday','Shift', 'Direction'], ascending=[True, True, True, True])

## 資料運算

1. 把班表資料黏上

In [14]:
ticketsoriginal = tickets.copy()


In [ ]:
# 這是我有提供資料的保證資料
tickets = tickets[tickets['RouteName'].isin(shift['RouteName'])].reset_index(drop = True)

# 開始對照表格